# INSTALLING DEPENDENCIES

In [ ]:
! pip install transformers==4.37.2 trl peft accelerate datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
  Attempting uninsta

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("TabbyML/SantaCoder-1B")

config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

In [ ]:
model

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49280, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTBigCodeBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear(in_features=2048, out_features=2304, bias=True)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear(in_features=8192, out_features=2048, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e

# Instruction Tuning - SFT

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from accelerate import Accelerator
import os

In [ ]:
sft_config = {
            "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "sft-tiny-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5
        }

In [ ]:
class TrainSFT:

    def __init__(self, data, config):
        self.data = data
        self.config = config

    def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)

    def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)

        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )

    def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )

    def train_and_save_model(self):
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])

In [ ]:
def create_data():
    data = load_dataset("tatsu-lab/alpaca", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:700]
    data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data = create_data()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_sft = TrainSFT(data, sft_config)
train_sft.train_and_save_model()

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.20437245579516677
None


...

adapter_model.safetensors:   0%|          | 0.00/9.02M [00:00<?, ?B/s]

events.out.tfevents.1708131457.81555dc0db8e.1614.0:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

# Preference Alignment - DPO/IPO/KTO-pair

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import DPOTrainer
import pandas as pd
from accelerate import Accelerator
from peft import get_peft_model, LoraConfig

In [ ]:
dpo_config = {
            "model_ckpt": "Vasanth/sft-tiny-chatbot",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 8,
            "lora_alpha": 8,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "tiny-chatbot-dpo",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "neftune_noise_alpha": 5,
            "beta": 0.1,
            "loss_type": "kto_pair",
            "max_length": 768,
            "max_prompt_length": 512,
            "max_target_length": 256,
            "is_encoder_decoder": False
        }

In [ ]:
class TrainDPO:

    def __init__(self, data, config):
        self.data = data
        self.config = config

    def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
        self.model_ref = get_peft_model(self.model_ref, self.lora_config)

    def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )

        self.model_ref = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)
        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )

    def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )
        else:
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        peft_config=self.lora_config,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )

    def train_and_save_model(self):
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])

In [ ]:
def create_data():
    df = load_dataset("Anthropic/hh-rlhf", split="train").to_pandas()
    df["prompt"] = df["chosen"].apply(lambda x: x.split("Assistant: ")[0])
    df["chosen"] = df["chosen"].apply(lambda x: "Assistant: "+ x.split("Assistant: ")[-1])
    df["rejected"] = df["rejected"].apply(lambda x: "Assistant: " + x.split("Assistant: ")[-1])
    df = df.sample(1000)
    data = Dataset.from_pandas(df)
    return data
data = create_data()

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dpo = TrainDPO(data, dpo_config)
train_dpo.train_and_save_model()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.10229075496156657
None


/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:328: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
100,0.502100
200,0.496300


# Inference

In [ ]:
from peft import AutoPeftModelForCausalLM
model = AutoPeftModelForCausalLM.from_pretrained("Vasanth/tiny-chatbot-dpo")
model

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=2048, out_features=256, bias=False)
             

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer="Vasanth/tiny-chatbot-dpo", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
A human can eat about 4-6 servings of helicopter food (typically made of meat, cheese, vegetables, fruits, and nuts) in a sitting, depending on their dietary preferences and the type of helicopter food they choose. However, consuming too many servings of helicopter food can lead to constipation, bloating, and digestive issues. It's always a good idea to choose helicopter food that is low in fat, calories, and sugar.
